In [14]:
pip install psycopg && pip install 'psycopg[binary]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 17.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [31]:
# делаем import необходимых библиотек
import os
import mlflow
import psycopg
import pandas as pd
from dotenv import load_dotenv

In [25]:
# подгружаем .env
load_dotenv()

connection = {"sslmode": "require", "target_session_attrs": "read-write"}

postgres_credentials = {
    "host": os.environ.get('DB_DESTINATION_HOST'), 
    "port": os.environ.get('DB_DESTINATION_PORT'),
    "dbname": os.environ.get('DB_DESTINATION_NAME'),
    "user": os.environ.get('DB_DESTINATION_USER'),
    "password": os.environ.get('DB_DESTINATION_PASSWORD'),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

In [28]:
# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

In [29]:
# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]


In [32]:
# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns) 

In [33]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,28173,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,None,0
1,28174,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0
2,28175,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,...,No,No,No,No,Male,0,No,No,No,1
3,28176,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,...,Yes,Yes,No,No,Male,0,No,No,None,0
4,28177,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,...,No,No,No,No,Female,0,No,No,No,1


In [38]:
# 1. Название колонок вашего датафрейма запишите в текстовый файл
os.makedirs('../data', exist_ok=True)
with open("../data/columns.txt", "w", encoding="utf-8") as fio:
    fio.write(', '.join(df.columns))

In [39]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

In [40]:
for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

		# обновите словарь stats
    stats.update(column_stat)

In [41]:
stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isnull().sum() # посчитайте кол-во пустых строк в колонке

In [42]:
stats

{'type_Month-to-month': 3875,
 'type_Two year': 1695,
 'type_One year': 1473,
 'paperless_billing_Yes': 4171,
 'paperless_billing_No': 2872,
 'internet_service_Fiber optic': 3096,
 'internet_service_DSL': 2421,
 'online_security_No': 3498,
 'online_security_Yes': 2019,
 'online_backup_No': 3088,
 'online_backup_Yes': 2429,
 'device_protection_No': 3095,
 'device_protection_Yes': 2422,
 'tech_support_No': 3473,
 'tech_support_Yes': 2044,
 'streaming_tv_No': 2810,
 'streaming_tv_Yes': 2707,
 'streaming_movies_No': 2785,
 'streaming_movies_Yes': 2732,
 'gender_Male': 3555,
 'gender_Female': 3488,
 'senior_citizen_0': 5901,
 'senior_citizen_1': 1142,
 'partner_No': 3641,
 'partner_Yes': 3402,
 'dependents_No': 4933,
 'dependents_Yes': 2110,
 'multiple_lines_No': 3390,
 'multiple_lines_Yes': 2971,
 'target_0': 5174,
 'target_1': 1869,
 'data_length': 7043,
 'monthly_charges_min': np.float64(18.25),
 'monthly_charges_max': np.float64(118.75),
 'monthly_charges_mean': np.float64(64.7616924605

In [43]:
df.to_csv("../data/users_churn.csv", index=False) 

In [44]:
# задаём название эксперимента и имя запуска для логирования в MLflow

EXPERIMENT_NAME = "churn_kruglikovAlex"
RUN_NAME = "data_check"

In [45]:
# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) # ваш код здесь

In [46]:
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id # ваш код здесь
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # объявлять переменную stats не надо,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats) # ваш код здесь
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact("../data/columns.txt", "dataframe") # ваш код здесь
    mlflow.log_artifact("../data/users_churn.csv", "dataframe") # ваш код здесь

In [47]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) # ваш код здесь

In [48]:
# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert run.info.status == "FINISHED" # ваш код здесь

In [49]:
run.info.status

'FINISHED'

In [50]:
run_id

'025243932bf248d6be1db9e80ad2f24a'

In [51]:
# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove("../data/columns.txt") # ваш код здесь
os.remove("../data/users_churn.csv") # ваш код здесь